In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import bs4
import re

## For a player

In [2]:
options = webdriver.EdgeOptions()
# options.add_argument('headless')
options.add_argument('inprivate')
driver = webdriver.Edge(options= options)

In [53]:
driver.get('https://qlstats.net/player/330308')

### Cookie screen

In [23]:
def is_cookie_screen(driver : selenium.webdriver):
    '''A function to check if the given webpage is the 'accept cookies' screen.
    Regex matches the body of the '''
    element = driver.find_element(By.TAG_NAME, value = 'body')
    cookie_screen = re.compile(r'.*(To continue using qlstats, you need to agree to the use of cookies.\nAgree).*')
    if cookie_screen.search(element.text) is None:
        return False
    return True


In [24]:
#press the button
if is_cookie_screen(driver):
    try:
        button = driver.find_element(By.TAG_NAME, 'button')
        button.click()
    except:
        "Cookies could not be accepted, please recheck"

### Scraping the player page

In [26]:
from bs4 import BeautifulSoup

In [55]:
soup = BeautifulSoup(driver.page_source, 'lxml')


In [56]:
# finding player name
player_name = soup.select_one('h2').text
print('Player name:\n{}'.format(player_name))

#for unavailable IDs:
if re.match(r"Sorry, that player wasn't found!", player_name):
    print("Player not found, can't proceed")

else:
    pass
    #TODO Scrape here

Player name:
BOOBISHA


In [57]:
# parse information from the p tab
p_tab_text = soup.select_one('p').text
p_tab_list = list(map(str.strip, p_tab_text.split("\n")))
p_tab_list = [x for x in p_tab_list if x != '']

p_tab_list

['Region: Europe',
 'Player ID: 330308',
 'Steam ID: 76561198062475631',
 'Joined: 2021-04-28   16:22:30',
 'Status: active']

In [ ]:
# press the overall button on the player page
button = driver.find_element(By.CLASS_NAME, 'tab-overall')
button.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".tab-overall"}
  (Session info: MicrosoftEdge=108.0.1462.54)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF78B9691D2+14306]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF78B8F0992+842866]
	(No symbol) [0x00007FF78B5A1D90]
	(No symbol) [0x00007FF78B5E5928]
	(No symbol) [0x00007FF78B5E5C10]
	(No symbol) [0x00007FF78B622507]
	(No symbol) [0x00007FF78B605D6F]
	(No symbol) [0x00007FF78B5D836E]
	(No symbol) [0x00007FF78B61F70C]
	(No symbol) [0x00007FF78B605B03]
	(No symbol) [0x00007FF78B5D73B8]
	(No symbol) [0x00007FF78B5D638E]
	(No symbol) [0x00007FF78B5D7AE4]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF78B83FE18+119032]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF78B82D4D6+42934]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x0

In [ ]:
for x in p_tab_list:
    

In [30]:
# parse winrate and elo info
soup.select('div', class_ = 'row tab-pane active')[1]

<div class="navbar-header">
<a class="navbar-toggle" data-target=".navbar-collapse" data-toggle="collapse">
<span class="icon-bar"></span>
<span class="icon-bar"></span>
<span class="icon-bar"></span>
</a>
<a class="navbar-brand" href="/">
<img alt="QL Logo" height="32" src="/static/css/img/menu_icon.png" style="float:left;margin-top:-5px;margin-right:10px;-webkit-filter: drop-shadow(0px 5px 5px grey);" width="32"/>
        QLStats
      </a>
</div>

In [31]:
soup.findAll('table', class_= 'table table-hover table-condensed')

[<table class="table table-hover table-condensed">
 <thead>
 <tr>
 <th>#</th>
 <th></th>
 <th>Played</th>
 <th>Type</th>
 <th>Server</th>
 <th>Map</th>
 <th>Result</th>
 <th>Opponent</th>
 <th>Rating</th>
 <th title="Rating ± Uncertainty">Old Glicko</th>
 <th title="Rating / Uncertainty">Glicko Change</th>
 </tr>
 </thead>
 <tbody>
 </tbody>
 </table>]

### Shut down the browser

In [32]:
# driver.close()